# Análise Exploratória: Incompatibilidades nos Dados

**Objetivo**: Identificar todos os casos de incompatibilidades nos dados de absentismo para discussão com RH.

Este notebook analisa incompatibilidades em todos os níveis da hierarquia de códigos:
- **Nível 1**: Presença vs Ausência no mesmo dia
- **Nível 2**: Categorias incompatíveis (ex: Presença + Ausência Justificada)
- **Nível 3**: Subcategorias incompatíveis (ex: Férias + Trabalho)
- **Nível 4**: Detalhes incompatíveis

**Output**: Excel com uma folha por tipo de incompatibilidade, contendo:
- Data
- Login do colaborador
- Nome do colaborador
- Código(s) em conflito

---

## 1. Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Carregar códigos (mesma estrutura do notebook de apresentação)
print('Carregando códigos...')
codigos = pd.read_excel('códigos.xlsx', engine='openpyxl')
codigos.columns = ['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']

# Limpar espaços
for col in codigos.columns:
    if codigos[col].dtype == 'object':
        codigos[col] = codigos[col].str.strip()

print(f'✅ Códigos carregados: {len(codigos)} códigos')
print(f'   Níveis: N1={codigos["nivel_1"].nunique()}, N2={codigos["nivel_2"].nunique()}, N3={codigos["nivel_3"].nunique()}, N4={codigos["nivel_4"].nunique()}')

# Carregar dataset
print('\nCarregando dataset...')
df_raw = pd.read_csv('combined_data.csv', sep=',', parse_dates=['Data'])

# Merge com códigos para ter todos os níveis
df_raw = df_raw.merge(
    codigos[['codigo', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']], 
    left_on='segmento_processado_codigo', 
    right_on='codigo', 
    how='left'
)
df_raw.drop('codigo', axis=1, inplace=True)

print(f'\n✅ Dataset carregado: {len(df_raw):,} registros')
print(f'   Período: {df_raw["Data"].min().strftime("%Y-%m-%d")} a {df_raw["Data"].max().strftime("%Y-%m-%d")}')
print(f'   Colaboradores únicos: {df_raw["login_colaborador"].nunique():,}')
print(f'   Colunas com níveis: {[c for c in df_raw.columns if "nivel" in c]}')

---

## 2. Identificação de Dias com Múltiplos Registros

In [ ]:
# Identificar dias com múltiplos registros
registros_por_dia = df_raw.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = registros_por_dia[registros_por_dia > 1]

print('='*90)
print('📊 DIAS COM MÚLTIPLOS REGISTROS')
print('='*90)
print(f'\nTotal de dias-colaborador únicos: {len(registros_por_dia):,}')
print(f'Dias com MÚLTIPLOS registros: {len(dias_duplicados):,} ({len(dias_duplicados)/len(registros_por_dia)*100:.1f}%)')
print(f'\nDistribuição:')
print(f'   1 registro:  {(registros_por_dia == 1).sum():,} dias')
print(f'   2 registros: {(registros_por_dia == 2).sum():,} dias')
print(f'   3 registros: {(registros_por_dia == 3).sum():,} dias')
print(f'   4+ registros: {(registros_por_dia >= 4).sum():,} dias')

# Filtrar apenas dias duplicados
dias_dup_list = dias_duplicados.index.tolist()
df_dup = df_raw.set_index(['login_colaborador', 'Data']).loc[dias_dup_list].reset_index()

print(f'\n✅ Dataset de dias duplicados criado: {len(df_dup):,} registros')

---

## 3. Análise de Incompatibilidades por Nível

### 3.1. Nível 1: Presença vs Ausência

In [ ]:
print('='*90)
print('🚨 NÍVEL 1: Presença E Ausência no mesmo dia')
print('='*90)

# Identificar dias onde há Presença E Ausência
conflitos_n1 = df_dup.groupby(['login_colaborador', 'Data'])['nivel_1'].apply(
    lambda x: x.nunique() > 1
)
dias_conflito_n1 = conflitos_n1[conflitos_n1].index.tolist()

# Criar DataFrame com detalhes
df_conflito_n1 = df_dup.set_index(['login_colaborador', 'Data']).loc[dias_conflito_n1].reset_index()

# Para cada dia conflituoso, criar uma linha com todos os códigos
df_n1_export = df_conflito_n1.groupby(['Data', 'login_colaborador', 'nome_colaborador']).agg({
    'segmento_processado_codigo': lambda x: ', '.join(sorted(set(x))),
    'nivel_1': lambda x: ' + '.join(sorted(set(x)))
}).reset_index()

df_n1_export.columns = ['Data', 'Login', 'Nome', 'Códigos', 'Conflito']
df_n1_export = df_n1_export.sort_values(['Data', 'Login'])

print(f'\n❌ Dias com Presença E Ausência: {len(dias_conflito_n1):,}')
print(f'   Registros afetados: {len(df_conflito_n1):,}')
print(f'   Colaboradores afetados: {df_conflito_n1["login_colaborador"].nunique():,}')
print(f'\n📋 Exemplos:')
print(df_n1_export.head(5).to_string(index=False))
print(f'\n✅ DataFrame criado: {len(df_n1_export)} linhas para exportar')

### 3.2. Nível 2: Categorias Incompatíveis

In [ ]:
print('='*90)
print('🚨 NÍVEL 2: Categorias Incompatíveis')
print('='*90)

# Definir combinações proibidas
combinacoes_proibidas_n2 = [
    ('Presença', 'Ausência Justificada'),
    ('Presença', 'Ausência Injustificada'),
    ('Presença', 'Inatividade'),
    ('Ausência Justificada', 'Ausência Injustificada'),
]

resultados_n2 = {}

for combo in combinacoes_proibidas_n2:
    # Encontrar dias com esta combinação
    conflitos = df_dup.groupby(['login_colaborador', 'Data'])['nivel_2'].apply(
        lambda x: set(combo).issubset(set(x))
    )
    dias_conflito = conflitos[conflitos].index.tolist()
    
    if len(dias_conflito) > 0:
        df_conflito = df_dup.set_index(['login_colaborador', 'Data']).loc[dias_conflito].reset_index()
        
        # Criar DataFrame para exportar
        df_export = df_conflito.groupby(['Data', 'login_colaborador', 'nome_colaborador']).agg({
            'segmento_processado_codigo': lambda x: ', '.join(sorted(set(x))),
            'nivel_2': lambda x: ' + '.join(sorted(set(x)))
        }).reset_index()
        
        df_export.columns = ['Data', 'Login', 'Nome', 'Códigos', 'Conflito']
        df_export = df_export.sort_values(['Data', 'Login'])
        
        nome_regra = f'{combo[0]} + {combo[1]}'
        resultados_n2[nome_regra] = df_export
        
        print(f'\n❌ {nome_regra}: {len(dias_conflito)} dias, {len(df_export)} casos')
        print(f'   Exemplos: {df_export.head(2).to_string(index=False)}')

print(f'\n✅ {len(resultados_n2)} tipos de conflitos encontrados no Nível 2')

### 3.3. Nível 3: Subcategorias Incompatíveis

In [ ]:
print('='*90)
print('🚨 NÍVEL 3: Subcategorias Incompatíveis')
print('='*90)

# Combinações proibidas no nível 3
combinacoes_proibidas_n3 = [
    ('Ferias / Feriado / Folga', 'Presença'),
    ('Ferias / Feriado / Folga', 'Atraso / Saída Antecipada'),
    ('Doença', 'Presença'),
    ('Maternidade / Paternidade', 'Presença'),
    ('Formação', 'Ausência Injustificada'),
]

resultados_n3 = {}

for combo in combinacoes_proibidas_n3:
    conflitos = df_dup.groupby(['login_colaborador', 'Data'])['nivel_3'].apply(
        lambda x: set(combo).issubset(set(x))
    )
    dias_conflito = conflitos[conflitos].index.tolist()
    
    if len(dias_conflito) > 0:
        df_conflito = df_dup.set_index(['login_colaborador', 'Data']).loc[dias_conflito].reset_index()
        
        df_export = df_conflito.groupby(['Data', 'login_colaborador', 'nome_colaborador']).agg({
            'segmento_processado_codigo': lambda x: ', '.join(sorted(set(x))),
            'nivel_3': lambda x: ' + '.join(sorted(set(x)))
        }).reset_index()
        
        df_export.columns = ['Data', 'Login', 'Nome', 'Códigos', 'Conflito']
        df_export = df_export.sort_values(['Data', 'Login'])
        
        nome_regra = f'{combo[0]} + {combo[1]}'
        resultados_n3[nome_regra] = df_export
        
        print(f'\n❌ {nome_regra}: {len(dias_conflito)} dias, {len(df_export)} casos')
        print(f'   Exemplos: {df_export.head(2).to_string(index=False)}')

print(f'\n✅ {len(resultados_n3)} tipos de conflitos encontrados no Nível 3')

### 3.4. Nível 4: Detalhes Incompatíveis

In [ ]:
print('='*90)
print('🚨 NÍVEL 4: Detalhes Incompatíveis')
print('='*90)

# Combinações proibidas no nível 4
combinacoes_proibidas_n4 = [
    ('Atraso', 'Ausência Injustificada'),
    ('Saída Antecipada', 'Ausência Injustificada'),
    ('Férias', 'Falta Injustificada'),
    ('Feriado', 'Falta Injustificada'),
    ('Baixa Médica', 'Presença Completa'),
]

resultados_n4 = {}

for combo in combinacoes_proibidas_n4:
    conflitos = df_dup.groupby(['login_colaborador', 'Data'])['nivel_4'].apply(
        lambda x: set(combo).issubset(set(x))
    )
    dias_conflito = conflitos[conflitos].index.tolist()
    
    if len(dias_conflito) > 0:
        df_conflito = df_dup.set_index(['login_colaborador', 'Data']).loc[dias_conflito].reset_index()
        
        df_export = df_conflito.groupby(['Data', 'login_colaborador', 'nome_colaborador']).agg({
            'segmento_processado_codigo': lambda x: ', '.join(sorted(set(x))),
            'nivel_4': lambda x: ' + '.join(sorted(set(x)))
        }).reset_index()
        
        df_export.columns = ['Data', 'Login', 'Nome', 'Códigos', 'Conflito']
        df_export = df_export.sort_values(['Data', 'Login'])
        
        nome_regra = f'{combo[0]} + {combo[1]}'
        resultados_n4[nome_regra] = df_export
        
        print(f'\n❌ {nome_regra}: {len(dias_conflito)} dias, {len(df_export)} casos')
        print(f'   Exemplos: {df_export.head(2).to_string(index=False)}')

print(f'\n✅ {len(resultados_n4)} tipos de conflitos encontrados no Nível 4')

---

## 4. Resumo de Todas as Incompatibilidades

In [ ]:
print('='*90)
print('📊 RESUMO GERAL DE INCOMPATIBILIDADES')
print('='*90)

total_casos = len(df_n1_export)
for df_temp in resultados_n2.values():
    total_casos += len(df_temp)
for df_temp in resultados_n3.values():
    total_casos += len(df_temp)
for df_temp in resultados_n4.values():
    total_casos += len(df_temp)

print(f'\nTotal de casos de incompatibilidades encontrados: {total_casos:,}')
print(f'\nDistribuição por nível:')
print(f'   Nível 1 (Presença/Ausência):      {len(df_n1_export):,} casos')
print(f'   Nível 2 (Categorias):             {sum(len(df) for df in resultados_n2.values()):,} casos')
print(f'   Nível 3 (Subcategorias):          {sum(len(df) for df in resultados_n3.values()):,} casos')
print(f'   Nível 4 (Detalhes):               {sum(len(df) for df in resultados_n4.values()):,} casos')

print(f'\n{'='*90}')
print('📁 Preparando exportação para Excel...')

---

## 5. Exportação para Excel

In [ ]:
# Criar arquivo Excel com múltiplas folhas
output_file = 'incompatibilidades_dados.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Nível 1
    df_n1_export.to_excel(writer, sheet_name='N1_Presença+Ausência', index=False)
    
    # Nível 2
    for i, (nome, df_temp) in enumerate(resultados_n2.items(), 1):
        sheet_name = f'N2_{nome[:25]}'  # Limitar tamanho do nome
        df_temp.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Nível 3
    for i, (nome, df_temp) in enumerate(resultados_n3.items(), 1):
        sheet_name = f'N3_{nome[:25]}'
        df_temp.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Nível 4
    for i, (nome, df_temp) in enumerate(resultados_n4.items(), 1):
        sheet_name = f'N4_{nome[:25]}'
        df_temp.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Criar folha de resumo
    resumo_data = {
        'Nível': [],
        'Tipo de Incompatibilidade': [],
        'Casos': [],
        'Folha': []
    }
    
    resumo_data['Nível'].append('1')
    resumo_data['Tipo de Incompatibilidade'].append('Presença + Ausência')
    resumo_data['Casos'].append(len(df_n1_export))
    resumo_data['Folha'].append('N1_Presença+Ausência')
    
    for nome, df_temp in resultados_n2.items():
        resumo_data['Nível'].append('2')
        resumo_data['Tipo de Incompatibilidade'].append(nome)
        resumo_data['Casos'].append(len(df_temp))
        resumo_data['Folha'].append(f'N2_{nome[:25]}')
    
    for nome, df_temp in resultados_n3.items():
        resumo_data['Nível'].append('3')
        resumo_data['Tipo de Incompatibilidade'].append(nome)
        resumo_data['Casos'].append(len(df_temp))
        resumo_data['Folha'].append(f'N3_{nome[:25]}')
    
    for nome, df_temp in resultados_n4.items():
        resumo_data['Nível'].append('4')
        resumo_data['Tipo de Incompatibilidade'].append(nome)
        resumo_data['Casos'].append(len(df_temp))
        resumo_data['Folha'].append(f'N4_{nome[:25]}')
    
    df_resumo = pd.DataFrame(resumo_data)
    df_resumo.to_excel(writer, sheet_name='RESUMO', index=False)

print(f'✅ Arquivo criado: {output_file}')
print(f'\nFolhas no Excel:')
print(f'   - RESUMO: Visão geral de todas as incompatibilidades')
print(f'   - N1_*: Incompatibilidades no Nível 1')
print(f'   - N2_*: Incompatibilidades no Nível 2')
print(f'   - N3_*: Incompatibilidades no Nível 3')
print(f'   - N4_*: Incompatibilidades no Nível 4')
print(f'\n📊 Total de folhas: {1 + len(resultados_n2) + len(resultados_n3) + len(resultados_n4) + 1}')
print(f'\n🎯 Próximo passo: Analisar com RH e decidir como tratar cada tipo de incompatibilidade')